In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import datetime
import matplotlib.dates as mdates

In [2]:
folder_path = '/Users/kassiewang/Downloads/csv/hashtags/'
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

dfs = []

for name in all_files:
    df = pd.read_csv(name)
    df['country'] = str(name[name.rfind('/')+1:-4])
    dfs.append(df)

hashtags = pd.concat(dfs)

In [3]:
#Getting list of economic terms
f = open('/Users/kassiewang/Desktop/econvocab.txt', 'r')
lines = f.readlines()
allterms = ''
for i in range(len(lines)):
    line = lines[i].strip()
    if i != 0:
        allterms = allterms + '|' + line
    else:
        allterms = line

feature = hashtags['text'].str.contains(allterms, case = False, na = False, regex = True)

In [4]:
hashtags['contain_econ'] = feature
hashtags['contain_econ'] = hashtags['contain_econ'].astype(int)
hashtags['date'] = hashtags['date'].str[:10]
hashtags['date'] = pd.to_datetime(hashtags['date'], errors='coerce')

In [5]:
hashtags['country']= hashtags['country'].replace('uk', "UK")
hashtags['country']= hashtags['country'].replace('italy', "Italy")
hashtags['country']= hashtags['country'].replace('eastern_us', "US")
hashtags['country']= hashtags['country'].replace('germany', "Germany")
hashtags['country']= hashtags['country'].replace('france', "France")
hashtags['country']= hashtags['country'].replace('spain', "Spain")
hashtags['country']= hashtags['country'].replace('switzerland', "Switzerland")
hashtags['country']= hashtags['country'].replace('south_korea', "South Korea")
hashtags['country']= hashtags['country'].replace('iran', "Iran")
hashtags['country']= hashtags['country'].replace('western_us', "US")

In [34]:
filepath = "/Users/kassiewang/Downloads/csv/county_coronavirus_cases.csv"
rawcorona = pd.read_csv(filepath, error_bad_lines=False)
rawcorona['Country.Region']= rawcorona['Country.Region'].replace('Mainland China', "China")

countries = ['US', 'Italy', 'Germany', 'Switzerland', 'France', 'Iran', 'Spain', 'UK', 'China', 'South Korea']
countriesstr = '|'.join(countries)

cleancorona = rawcorona[rawcorona['Country.Region'].str.contains(countriesstr, case = True, regex = True)]
cleancorona['ObservationDate'] = pd.to_datetime(cleancorona['ObservationDate'])
cleancorona.rename(columns = {'ObservationDate':'date'}, inplace = True)
             

b'Skipping line 3: expected 5 fields, saw 6\nSkipping line 276: expected 5 fields, saw 6\nSkipping line 277: expected 5 fields, saw 6\nSkipping line 278: expected 5 fields, saw 6\nSkipping line 279: expected 5 fields, saw 6\nSkipping line 1193: expected 5 fields, saw 6\nSkipping line 1194: expected 5 fields, saw 6\nSkipping line 1195: expected 5 fields, saw 6\nSkipping line 1196: expected 5 fields, saw 6\nSkipping line 1197: expected 5 fields, saw 6\n'
/Users/kassiewang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/kassiewang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on

In [37]:
bydatetweets = hashtags.groupby(['date'],as_index=False).agg({'contain_econ': 'sum'})
bydatecorona = cleancorona.groupby(by='date').sum()
finaldf = pd.merge(bydatetweets, bydatecorona, how='outer', on = 'date')
finaldf = finaldf.sort_values(by=['date'])
finaldf.to_csv('/Users/kassiewang/Desktop/coronatweet_features.csv')

,date,contain_econ
0,2020-01-30,17
1,2020-01-31,6
2,2020-02-01,2
3,2020-02-02,1
4,2020-02-03,1
